In [1]:
from transformers import DonutProcessor
import sys
sys.path.insert(1, 'PubTabNet/src/')

from metric import TEDS
import json
import os


SPLIT = "train/"

PUBTABNET = 'anns/'

ANN_PATH = PUBTABNET

json_list = os.listdir(ANN_PATH + SPLIT)

aux_list = []
html_list = []

for f in json_list:
    if(f[-6:-5] == "L"):
        html_list.append(f)
    else:
        aux_list.append(f)
json_list = aux_list
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")

2023-12-15 13:09:33.874960: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-15 13:09:33.874987: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-15 13:09:33.875004: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-15 13:09:34.548533: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [2]:
from transformers import AddedToken

new_tokens = ["<table_extraction>", "<table>", "<row>", "<cell>", "<row_and_col_header>", "<row_header>", "<col_header>"]
new_tokens += ["<content_row_and_col_header>", "<content_row_header>", "<content_col_header>", "<content>"]

for i in range(2):
    for j in range(2):
        for k in range(2):
            new_tokens.append("<span_type=0" + str(i) + str(j) + str(k) + ">")
            new_tokens.append("<span_type=1" + str(i) + str(j) + str(k) + ">")

#processor.tokenizer.add_tokens(new_tokens, special_tokens = False)

In [3]:
from tqdm.auto import tqdm

content_list = []

for i, file_name in enumerate(tqdm(json_list)):
    with open(ANN_PATH + SPLIT + file_name, encoding="utf-8") as f:
            annotation = json.load(f)
    for row in annotation['tables'][0]:
        for cell in row:
            content_list.append(cell['content'])

  0%|          | 0/500777 [00:00<?, ?it/s]

In [4]:
len(processor.tokenizer)

57525

In [5]:
from transformers import AddedToken

new_tokenizer = processor.tokenizer.train_new_from_iterator(
    content_list,
    length = len(content_list),
    vocab_size = 8000,
    show_progress = True)

processor.tokenizer = new_tokenizer

In [6]:
new_tokenizer

XLMRobertaTokenizerFast(name_or_path='naver-clova-ix/donut-base', vocab_size=8000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['<s_iitcdip>', '<s_synthdog>']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
	

In [7]:
processor.tokenizer.decode(processor.tokenizer("gfhgf <i> sadsd").input_ids)

'<s> gfhgf <i> sadsd</s>'

In [8]:
processor.tokenizer("Guine-Bisau").input_ids

[0, 4080, 211, 1972, 270, 1985, 2]

In [9]:
processor.tokenizer("<cell> zxzcxzc").input_ids

[0, 94, 2590, 184, 8, 425, 160, 425, 86, 160, 425, 86, 2]

In [10]:
processor.tokenizer.decode([4])

'<mask>'

In [11]:
for i in range(4000, 4500):
    print(processor.tokenizer.decode([i]))

single
field
0.032
diff
0.36
what
pine
0.031
ono
Less
Mar
Log
ENT
self
Hydro
<b>Per
β-
Care
unknown
unt
bp
19.8
anti
yo
(0.95
EB
CCTT
following
Respon
(−
1.78
Central
-to-
Ang
agin
treated
/H
19.6
<b>Education
n.
(0.04
HPV
Mouse
mRNA
BG
behaviour
lock
19.7
23.5
features
190
0.90
Adult
18.9
sleep
Glyc
iter
19.3
fr
distribution
174
1.74
<i>N</i>
dec
<i>t</i>
<i>P</i>-value
176
pT
€
aid
178
(0.94
link
lapse
More
DF
Phase
0.38
19.1
(°C)
Gu
0.12
kill
depression
</i>value</b>
TTTG
0.15
born
/l
GTTG
<b>Range
EU
<b>95
role
gy
head
van
1.71
>3
flex
(0.09
follow
(bp)
(2.0)
−0.8
–2.5
hood
Surgery
task
American
0.39
1.77
super
-type
ki
UD
nm
code
(50.0)
genotype
e.g.
0.035
ATP
trop
(median
position
<b>Effect
performance
small
0.93
men
State
diameter
-h
Sal
mono
LDL
gal
Glucose
<b><i>N</i></b>
Mild
moderate
23.8
Symptom
TO
included
LY
(27)
aria
tub
tile
GH
Random
intra
random
Systolic
main
Drug
173
23.0
ich
three
After
KD
minutes
179
177
(1.8)
hel
measure
cut
<b>CD
K1
UR
doctor
(0.98
production
(2.

In [12]:
processor.save_pretrained("Donut_PubTables_TML_Processor8k")

In [8]:
json_list[:10]

['PMC1637031_008_00.json',
 'PMC5630437_002_00.json',
 'PMC2783011_003_00.json',
 'PMC3019163_006_00.json',
 'PMC3598484_002_00.json',
 'PMC2292170_004_01.json',
 'PMC2408603_007_00.json',
 'PMC3568023_005_01.json',
 'PMC3102902_004_01.json',
 'PMC5637583_001_00.json']

In [3]:
def cel2token(cell):
    if cell['span_type'][10:] != '0000':
        sequence = "<" + cell['span_type'] + ">"
        if cell['content_holder']:
            if cell['row_header'] and cell['col_header']:
                sequence += "<content_row_and_col_header>"
            elif cell['col_header']:
                sequence += "<content_col_header>"
            elif cell['row_header']:
                sequence += "<content_row_header>"
            else:
                sequence += "<content>"
            sequence += cell['content']
    else:
        sequence = ""
        if cell['content_holder']:
            if cell['row_header'] and cell['col_header']:
                sequence += "<row_and_col_header>"
            elif cell['col_header']:
                sequence += "<col_header>"
            elif cell['row_header']:
                sequence += "<row_header>"
            else:
                sequence += "<cell>"
            sequence += cell['content']
        
    return sequence

def row2token(row):
    sequence = "<row>"
    for cell in row:
        sequence += cel2token(cell)
    
    return sequence


def table2token(table):
    sequence = "<table>"
    for row in table:
        sequence += row2token(row)
    
    return sequence


def json2token(json):
    sequence = ""
    if('tables' in json):
        for table in json['tables']:
            sequence += table2token(table)

    return sequence

In [4]:
cell_types = ["<cell>", "<col_header>", "<row_header>", "<row_and_col_header>"]
content_types = ["<content_row_and_col_header>", "<content_row_header>", "<content_col_header>", "<content>"]


added_content = ["<b>", "<i>", "<sup>", "<sub>","</b>", "</i>", "</sup>", "</sub>"]

for i in range(2):
    for j in range(2):
        for k in range(2):
            cell_types.append("<span_type=0" + str(i) + str(j) + str(k) + ">")
            cell_types.append("<span_type=1" + str(i) + str(j) + str(k) + ">")


def token2cell(processor, seq, i, cell_coord, cell_type):
    if cell_type[:5] == "<span": 
        span_type = cell_type[1:-1]
        cell_type = processor.decode(seq[i])
        if cell_type[:5] == "<cont":
            i += 1
            if cell_type != "<content>":
                cell_type = "<"+cell_type[9:]
            else:
                cell_type = "<cell>"
    else:
        span_type = "span_type=0000"
    
    x, y = cell_coord
    aux_cell = {
        "row": x,
        "col": y,
        "col_header": cell_type in ["<col_header>", "<row_and_col_header>"],
        "row_header": cell_type in ["<row_header>", "<row_and_col_header>"],
        "colspan": -1 if span_type != "span_type=0000"  else 1,
        "rowspan": -1 if span_type != "span_type=0000"  else 1,
        "span_type": span_type,
        "content": ""
    }
    start_cont = i
    while processor.decode(seq[i]) not in ["<row>", "</s>", "<table>"] + cell_types:
        i += 1
        
    aux_cell['content'] += processor.decode(seq[start_cont: i])
    
    return aux_cell, i

def token2row(processor, seq, i, row_id):
    cells = []
    while processor.decode(seq[i]) not in ["<row>", "</s>", "<table>"]:
        if processor.decode(seq[i]) in cell_types:
            aux_cell, i = token2cell(processor, seq, i+1, (row_id, len(cells)), processor.decode(seq[i]))
            cells.append(aux_cell)
        else:
            i += 1
    return cells, i


def token2table(processor, seq, i):
    rows = []
    while processor.decode(seq[i]) not in ["</s>", "<table>"]:
        if processor.decode(seq[i]) == "<row>":
            aux_row, i = token2row(processor, seq, i+1, len(rows))
            rows.append(aux_row)
        else:
            i += 1
            
    return rows, i


def crop_empty_left(table):
    for row in table:
        empty_left = []
        for cell in row:
            if(cell['content'] == "" and cell['colspan'] == 1):
                empty_left.append(cell)
            else:
                empty_left.clear()
        
        for cell in empty_left:
            cell['colspan'] = 0
            cell['rowspan'] = 0

def token2ann(processor, seq, i):
    tables = []
    while processor.decode(seq[i]) != "</s>":
        if processor.decode(seq[i] == "<table>"):
            aux_table, i = token2table(processor, seq, i+1)
            #crop_empty_left(aux_table)
            tables.append(aux_table)
        else:
            i += 1
        
    return {'tables': tables}

In [5]:
def update_vals(cell, content, col_header, row_header):
    cell['colspan'] = 0
    cell['rowspan'] = 0
    
    if cell['content'] != "":
        content = cell['content']
    
    col_header = col_header or cell['col_header']
    row_header = row_header or cell['row_header']
    
    return content, col_header, row_header


def define_by_path(cell, table):
    col_header, row_header = False, False
    i, j = cell['row'], cell['col']
    first_j = j

    content = ''
    
    while True:
        while True:
            content, col_header, row_header = update_vals(cell, content, col_header, row_header)
            
            if(cell['span_type'][-4:-3] != '1'):
                break
            j += 1
            cell = table[i][j]
            
        if(cell['span_type'][-3:-2] != '1'):
            break
        i += 1
        j = first_j
        cell = table[i][j]
    
    return (i, j), content, col_header, row_header

    
def define_spannings(table):
    for row in table:
        for cell in row:
            if(cell['colspan'] == -1):
                end_coord = (cell['row'], cell['col'])
                content = cell['content']
             
                end_coord, content, col_header, row_header = define_by_path(cell, table)
                
                
                cell['rowspan'] = end_coord[0] - cell['row'] + 1
                cell['colspan'] = end_coord[1] - cell['col'] + 1
                cell['col_header'] = col_header
                cell['row_header'] = row_header
                cell['content'] = content
                

In [6]:
def cell2html(cell):
    seq = "<td"
    if(cell['rowspan'] > 1):
        seq += ' rowspan="' + str(cell['rowspan']) +'"'
    if(cell['colspan'] > 1):
        seq += ' colspan="' + str(cell['colspan']) +'"'
    seq += ">" + cell["content"] + "</td>"
    
    return seq

def table2html(table):
    seq = ""
    head = False
    body = False
    for row in table:
        row_seq = ""
        count_header = 0
        row_len = len(row)
        for cell in row:
            if(cell['colspan'] > 0):
                count_header += cell['col_header']
                row_seq += cell2html(cell)
            else:
                row_len -= 1
            
        if(count_header  > row_len/2):
            if body:
                seq += "</tbody>"
                body = False
                
            if not head:
                seq += "<thead>"
                head = True
            seq +=  "<tr>" + row_seq + "</tr>" 
        else:
            if head:
                seq += "</thead>"
                head = False
                
            if not body:
                seq += "<tbody>"
                body = True
                
            seq +=  "<tr>" + row_seq + "</tr>"
    
    if head:
        seq += "</thead>"
        
    if body:
        seq += "</tbody>"
    
    return seq

In [7]:
gt_list = []
pred_list = []

teds = TEDS(n_jobs=4, structure_only = False)

In [8]:
processor.tokenizer("</s>")

{'input_ids': [0, 2, 2], 'attention_mask': [1, 1, 1]}

In [10]:
import random
import torch
from difflib import SequenceMatcher
from tqdm.auto import tqdm

removed_tags = []
soma_ratios = 0

for i, file_name in enumerate(tqdm(json_list)):

    with open(ANN_PATH + SPLIT + file_name, encoding="utf-8") as f:
        annotation = json.load(f)
    with open(ANN_PATH + SPLIT + file_name[:-5]+"-HTML.json", encoding="utf-8") as f:
        gt_html = "<html><body><table>"+str(json.load(f))+"</table></body></html>"
    
    for tag in removed_tags:
        gt_html = "<html><body><table>"+gt_html.replace(tag, "")+"</table></body></html>"
    
    target_sequence = "<s>"+json2token(annotation)+"</s>"
    input_ids = processor.tokenizer(
        target_sequence,
        add_special_tokens=False,
        max_length= 4096,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"].squeeze(0)
    
    
    input_ids = torch.cat((input_ids, torch.Tensor([2, 2]).int()), 0)
    
    table = token2ann(processor, input_ids, 1)
    table_aux  = table.copy()
    
    
    
    define_spannings(table['tables'][0])
    pred_html = "<html><body><table>"+table2html(table['tables'][0])+"</table></body></html>"
    
    pred_html = pred_html
    gt_html = gt_html
    
    pred_list.append(pred_html)
    gt_list.append(gt_html)
    
    ratio = teds.evaluate(str(pred_html), str(gt_html))
    if(ratio < .9):
        print(processor.decode(input_ids))
        #print(target_sequence)
        #print(table['tables'][0])
        print(pred_html)
        print(gt_html)
    if i% 10 == 0:   
        print(str(i)+'->', soma_ratios/max(1, i))
    
    soma_ratios += ratio
print(soma_ratios/len(json_list))

  0%|          | 0/500777 [00:00<?, ?it/s]

0-> 0.0
<s><table><row><span_type=1000><span_type=0001><content_col_header> <b> </b><col_header> <b>Baseline meaning in life scores</b><col_header> <b>Follow-up meaning in life scores</b><row><span_type=1000><span_type=0001><content> Stable non-smokers (a)<cell> 10.21 (2.57)<cell> 10.02 (2.73)<row><span_type=1000><span_type=0001><content> Stable smokers (b)<cell> 9.66 (2.70)<cell> 9.54 (2.92)<row><span_type=1000><span_type=0001><content> Starters (c)<cell> 9.81 (2.62)<cell> 9.78 (2.85)<row><span_type=1000><span_type=0001><content> Quitters (d)<cell> 9.85 (2.66)<cell> 9.74 (3.05)<row><span_type=1000><span_type=0001><content> Kruskall-Wallis one-way analysis of ranks<cell> <i>χ</i><sup>2</sup> = 35.11<cell> <i>χ</i><sup>2</sup> = 19.61<row><cell> <cell> <cell> df = 3<cell> df = 3<row><cell> <cell> <cell> p < 0.001<cell> p < 0.001<row><cell> Multiple pairwise comparisons with Mann–Whitney test<cell> a vs. b<cell> U = 1236219.5; p < 0.001; r = 0.09<cell> U = 1306015.0; p < 0.001; r = 0.07<

10-> 0.9832179096075799
20-> 0.9907650649886108
30-> 0.9895669406947921
<s><table><row><span_type=0100><span_type=0100><span_type=1000><span_type=0001><content_col_header> <b>Post-operatively DTC status</b><span_type=0100><row><span_type=0010><content_col_header> <b>Variable</b><span_type=0010><content_col_header> <b>No. of patients n = 154</b><col_header> <b>Positive n = 23</b><col_header> <b>Negative n = 131</b><span_type=0010><content_col_header> <b>P-values</b><row><cell> <b>Age</b><cell> <cell> <cell> <cell> <b>0.013</b><row><cell> <=55 years<cell> 78<cell> 6 (8)<cell> 72 (92)<cell> <row><cell> >55 years<cell> 76<cell> 17 (22)<cell> 59 (78)<cell> <row><cell> <b>Lymph node status</b><cell> <cell> <cell> <cell> 0.223<row><cell> pN0<cell> 110<cell> 14 (13)<cell> 96 (87)<cell> <row><cell> pN1-2<cell> 44<cell> 9 (20)<cell> 35 (80)<cell> <row><cell> <b>Tumor size</b><cell> <cell> <cell> <cell> 0.469<row><cell> pT<unk><cell> 105<cell> 14 (13)<cell> 91 (87)<cell> <row><cell> pT2-4<cell> 4

40-> 0.9805365293061838
<s><table><row><span_type=0100><span_type=1000><span_type=1001><span_type=0001><content_col_header> <b>SBP</b><span_type=1000><span_type=1001><span_type=0001><content_col_header> <b>DBP</b><row><span_type=0010><content_col_header> <b>Pollutant</b><col_header> <b>β (SE)</b><col_header> <b>t-Value</b><col_header> <b>β (SE)</b><col_header> <b>t-Value</b><col_header><col_header><row><cell> Odor (0–8)<cell> 0.10 (0.12)<cell> 0.86<cell> 0.23 (0.08)<cell> 3.02<cell><cell><row><cell> H2S (ppb)<cell> 0.29 (0.12)<cell> 2.45<cell> 0.12 (0.08)<cell> 1.52<cell><cell><row><cell> PM10 (μg/m3)<cell> –0.01 (0.01)<cell> –0.78<cell> –0.00 (0.01)<cell> –0.41<cell><cell><row><cell> Semivolatile PM10 (μg/m3)<cell> –0.02 (0.05)<cell> –0.45<cell> –0.06 (0.03)<cell> –<unk>.66<cell><cell></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

50-> 0.9788662163133662
<s><table><row><col_header><span_type=1000><span_type=1001><span_type=1001><span_type=1001><span_type=1001><span_type=1001><span_type=1001><span_type=0001><content_col_header> <b>Display size</b><row><col_header><span_type=1000><span_type=0001><content_col_header> <b>Experiment 1</b><span_type=1000><span_type=0001><content_col_header> <b>Experiment 2a</b><span_type=1000><span_type=0001><content_col_header> <b>Experiment 2b</b><span_type=1000><span_type=0001><content_col_header> <b>Experiment 3</b><row><col_header><col_header> <b>6</b><col_header> <b>8</b><col_header> <b>6</b><col_header> <b>8</b><col_header> <b>6</b><col_header> <b>8</b><col_header> <b>6</b><col_header> <b>8</b><row><cell> <b>Distractor condition</b><cell><cell><cell><cell><cell><cell><cell><cell><row><cell> No distractor<cell> 0.011<cell> 0.014<cell> 0.016<cell> 0.024<cell> 0.008<cell> 0.008<cell> 0.019<cell> 0.022<row><cell> Compatible distractor<cell> 0.003<cell> 0.007<cell> 0.008<cell> 0.014

...

440-> 0.985694641495152
450-> 0.9858616405608183
460-> 0.9861009526174055
470-> 0.9862499618848949
480-> 0.9864724150238433
490-> 0.986540862968255
500-> 0.9866878778141451
510-> 0.9868557822800755
520-> 0.9870309423903542
530-> 0.987241808791311
540-> 0.9873755233513394
550-> 0.9874240144028287
560-> 0.9875513584657473
570-> 0.9874725339536294
<s><table><row><span_type=0100><span_type=1000><span_type=0001><content_col_header> <b>HCMECs</b><col_header><row><span_type=0110><col_header> <b>Young</b><col_header> <b>Young and <i>O. sanctum</i> Linn. extract</b><col_header><row><span_type=0010><content_col_header> <b>Cell conditions</b><cell> ChAT expressions<cell> +++<cell> +++</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

580-> 0.9874047253207991
590-> 0.987531421663742
<s><table><row><col_header> <b>Symptom</b><col_header> <b>H<unk>N<unk> Seroconversion n = 80 (%)</b><col_header> <b>Tested Negative for H<unk>N<unk> n = 159 (%)</b><col_header> <b>P-value</b><row><cell> Fever<cell> 53 (66.3%)<cell> 36 (22.6%)<cell> <.0001*<row><cell> Sore Throat<cell> 37 (56.9%)<cell> 48 (36.9%)<cell>.0094*<row><cell> Cough<cell> 59 (83.1%)<cell> 104 (72.7%)<cell>.1246<row><cell> Headache<cell> 31 (50.8%)<cell> 26 (21.3%)<cell> <.0001*<row><cell> Vomiting<cell> 17 (32.1%)<cell> 11 (9.9%)<cell>.0007*<row><cell> Running Nose<cell> 55 (82.1%)<cell> 113 (80.7%)<cell>.8520<row><cell> Stomach upset<cell> 10 (20.0%)<cell> 11 (9.8%)<cell>.0824<row><cell> Demographic variables<cell> <cell> <cell> <row><cell> Gender<cell> <cell> <cell>.1657<row><cell> Male<cell> 34 (43.0%)<cell> 82 (53.6%)<cell> <row><cell> Female<cell> 45 (57.0%)<cell> 71 (46.4%)<cell> <row><cell> Grade<cell> <cell> <cell>.0172<row><cell> Low (1–3)<cell> 39 (48.8

600-> 0.9875061296324733
610-> 0.9874674758425128
620-> 0.9876214216648111
630-> 0.9877973859871212
640-> 0.9879056672033972
<s><table><row><col_header> <b>Characteristics</b><col_header> <b>Case tested</b><span_type=1000><span_type=0001><content_col_header> <b>Positive tests</b><row><col_header> <b> </b><col_header> <b> </b><col_header> <b>n</b><col_header> <b>%</b><row><cell> Immunohistochemistry<cell> <cell> <cell> <row><cell> CgA<cell> 149<cell> 103<cell> 69.<unk><row><cell> Syn<cell> 143<cell> 129<cell> 90.2<row><cell> Tumor grading<cell> <cell> <cell> <row><cell> G<unk><cell> 169<cell> 87<cell> 51.5<row><cell> G2<cell> 169<cell> 31<cell> 18.3<row><cell> G3<cell> 169<cell> 51<cell> 30.2<row><cell> Tumor type<cell> <cell> <cell> <row><cell> NET<cell> 169<cell> 118<cell> 69.8<row><cell> NEC<cell> 169<cell> 46<cell> 27.2<row><cell> MANEC<cell> 169<cell> 5<cell> 3.0<row><cell> Infiltration/Metastasis<cell> <cell> <cell> <row><cell> Muscularis/Serosa infiltration<cell> 100<cell> 53<cel

660-> 0.9876150333650813


KeyboardInterrupt: 

In [ ]:
input_ids

In [ ]:
json.dump(pred_list, open("teste-pred.json", 'w'))
json.dump(gt_list, open("teste-gt.json", 'w'))

In [ ]:
a = '<thead><tr><td rowspan="2"><b>Cutoff (IRS)</b></td><td rowspan="2"><b>Number</b></td><td colspan="2"><b>Overall survival prognosis</b></td></tr><tr><td><b>Log-rank <i>P</i></b></td><td><b>Corrected <i>P</i><sup><i>∗</i></sup></b></td></tr></thead><tbody><tr><td>≥1</td><td>99/94</td><td>5.56</td><td>0.0184</td></tr><tr><td>≥2</td><td>129/64</td><td>4.37</td><td>0.0377</td></tr><tr><td>≥3</td><td>150/43</td><td>8.37</td><td>0.0038</td></tr><tr><td>≥4</td><td>161/32</td><td>6.35</td><td>0.0117</td></tr><tr><td>≥5 </td><td>177/16</td><td>4.50</td><td>0.0338</td></tr></tbody>'

for tag in ["<i>", "</i>","<b>","</b>"]:
    a = a.replace(tag, "")
    
a

In [ ]:
target_sequence = "Capítulo Primeiro Do título Uma noite destas, vindo da cidade para o Engenho Novo, encontrei no[ 1 ] trem da Central um rapaz aqui do bairro, que eu conheço de vista e de cha- péu. Comprimentou-me, sentou-se ao pé de mim, falou da lua e dos mi- nistros, e acabou recitando-me versos. A viagem era curta, e os versos pode ser que não fossem inteiramente maus. Succedeu, porém, que, como eu es- tava cansado, fechei os olhos três ou quatro vezes; tanto bastou para que ele interrompesse a leitura e metesse os versos no bolso. — Continue, disse eu acordando.[ 2 ] — Já acabei, murmurou ele.[ 3 ] — São muito bonitos.[ 4 ] Vi-lhe fazer um gesto para tirá-los outra vez do bolso, mas não passou[ 5 ] do gesto; estava amuado. No dia seguinte entrou a dizer de mim nomes feios, e acabou alcunhando-me Dom Casmurro."

input_ids = processor.tokenizer(
        target_sequence,
        add_special_tokens=True,
        max_length= 1536,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"].squeeze(0)

In [ ]:
input_ids

In [ ]:
for token in input_ids:
    print(processor.decode(token))

In [ ]:
processor.decode(input_ids)